In [19]:
import tensorflow as tf
import numpy as np
from datetime import *

In [20]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [21]:
# input vectors
x = tf.placeholder(tf.float32, [None, 784], name='x')

In [22]:
# model parameters
with tf.name_scope('parameters'):
    W = tf.Variable(tf.zeros([784, 10]), name='W')
    b = tf.Variable(tf.zeros([10]), name='b')
    y = tf.nn.softmax(tf.matmul(x, W) + b, name='Wx_b')

In [23]:
# training
with tf.name_scope('loss'):
    # truth
    y_ = tf.placeholder(tf.float32, [None, 10])
    # loss function
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    # optimizer
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [24]:
# run 
with tf.name_scope('training'):
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    for _ in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [25]:
# eval
with tf.name_scope('scoring'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9115


In [27]:
# saving model
saver = tf.train.Saver()
save_path = saver.save(sess, "./model/model.ckpt")
print("Model saved in file: %s" % save_path)

Model saved in file: ./model/model.ckpt


In [28]:
merged = tf.summary.merge_all()
log_path = datetime.now().strftime('./logs/%m-%d_%H_%M_%S')
file_writer = tf.summary.FileWriter(log_path, sess.graph)
sess.close()